# Calculate ICC of QAP measures from CORR data
Code to calculate the modern ICC version (using linear mixed-effects model).
A lot is borrowed from AFNI's 3dICC_REML.R. Big shout out to the AFNI folks.

In [1]:
library(lme4)
library(dplyr)
library(reshape2)
library(tidyr)
library(ggplot2)
library(grid)
library(gridExtra)

Loading required package: Matrix

Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Attaching package: ‘tidyr’

The following object is masked from ‘package:reshape2’:

    smiths

The following object is masked from ‘package:Matrix’:

    expand


Attaching package: ‘gridExtra’

The following object is masked from ‘package:dplyr’:

    combine



In [2]:
#' ## ICC
#' 
#' For each measure and site, we calculate the ICC.
#' We do this with our own code so we can ensure non-negative values amongst other things.
#' 
#+ calc-icc
calc_icc <- function(x) {
  # ICC Juicy Core
    icc <- function(.) {
        s_e = sigma(.)
        s_a = unname(s_e * getME(., "theta"))
        s_a^2 / (s_e^2 + s_a^2)
    }

    fmAOV <- lmer(value ~ 1 + Site + (1|Participant), sdf)

#     system.time(boo01 <- bootMer(fmAOV, icc, nsim = 100))
}

## Read and tidy the CORR anatomical QAP measures

In [3]:
# read the data
corr_anat_qa<-read.csv("2016_05_CORR_qap_anatomical_spatial.csv")

# reduce to the columns that we are interested in
id.vars=c('Participant','Site','Session','Series')
measure.vars=c('CNR','Cortical.Contrast','EFC','FBER','FWHM','Qi1','SNR')

corr_anat_qa<-corr_anat_qa[c(id.vars,measure.vars)]

# Filter 1
# Use only the first scan from sessions 1 & 2
corr_anat_qa <-  corr_anat_qa %>% 
                  filter(Series == 'anat_1') %>% 
                   filter(Session %in% c('session_1','session_2'))
corr_anat_qa$Series <- droplevels(corr_anat_qa$Series)
corr_anat_qa$Session <- droplevels(corr_anat_qa$Session)

# Filter 2 
# remove participants that do not have two sessions of data
pts_keep <- (corr_anat_qa %>% 
             group_by(Participant) %>% 
              summarise(N=n()) %>%
               filter(N>1))$Participant
corr_anat_qa <-  corr_anat_qa %>% 
                  filter(Participant %in% pts_keep)
# corr_anat_qa$Participant <- droplevels(corr_anat_qa$Participant)

# Filter 3 
# remove Sites that have fewer than 5 pts
sites_keep <- (corr_anat_qa %>% 
                 group_by(Site,Session) %>% 
                   summarise(N=n()) %>%
                     filter(N>5))$Site

corr_anat_qa <-  corr_anat_qa %>% 
                  filter(Site %in% sites_keep)
corr_anat_qa$Site <- droplevels(corr_anat_qa$Site)

corr_anat_qa <- corr_anat_qa %>% 
                  melt(id.vars=id.vars,
                       measure.vars=measure.vars,
                       variable.name="Measure")

corr_anat_qa$Participant <- factor(corr_anat_qa$Participant)

table(corr_anat_qa$Site, corr_anat_qa$Session)

         
          session_1 session_2
  BNU_1         350       350
  BNU_2         427       427
  HNU_1         210       210
  IACAS         189       189
  IBA_TRT        98        98
  IPCAS_1       210       210
  IPCAS_2       245       245
  IPCAS_8        91        91
  LMU_3         175       175
  MRN           336       336
  NYU_1         175       175
  NYU_2         434       434
  SWU_1         133       133
  SWU_3         168       168
  SWU_4        1638      1638
  UM            560       560
  UPSM_1        693       693
  Utah_1        133       133
  UWM           175       175
  XHCUMS        168       168

In [ ]:
library(boot)
sdf<-corr_anat_qa %>% 
       filter(Measure == 'FWHM') %>% 
         spread(Session, value)

icc <- function(data, indices) {
  d <- data[indices,] %>% 
         melt(id.vars=c('Participant','Site',
                        'Series','Measure'),
              measure.vars=c('session_1','session_2'),
                        variable.name="Session")
  fit <- lmer(value ~ 1 + Site + (1|Participant), d)
  s_e = sigma(fit)
  s_a2 = VarCorr(fit)[[1]][1]
  return(s_a2 / (s_e^2 + s_a2))
}

system.time(results <- boot(data=sdf, statistic=icc, R=1000))

In [ ]:
# sdf<-corr_anat_qa %>% 
#        filter(Measure == 'FWHM') %>% 
#          spread(Session, value)

# icc <- function(data, indices) {
#   d <- data[indices,] %>% 
#          melt(id.vars=c('Participant','Site',
#                         'Series','Measure'),
#               measure.vars=c('session_1','session_2'),
#                         variable.name="Session")
#   fit <- lmer(value ~ 1 + Site + (1|Participant), d)
#   s_e = sigma(fit)
#   s_a2 = VarCorr(fit)[[1]][1]
#   return(s_a2 / (s_e^2 + s_a2))
# }

# bootstrap_ci <- function(nboot, data, boot_fun) {
#     data_len = nrow(sdf)
#     resamples <- lapply(1:nboot, function(i) sample(data_len, replace = T))
#     t0<-boot_fun(sdf,seq(1,data_len))
#     t_bs<-vector(mode="numeric",length=nboot)
        
#     for( i in 1:length(resamples) ){
#         t_bs[i] = icc(sdf,resamples[[i]])
#         if (i %% 100 == 0){
#             print(sprintf("%d %lf",i,t_bs[i]))
#         }
#     }
    
#     return(list(t0=t0,t_bs=t_bs))
# }
# system.time(tt <- bootstrap_ci(100,sdf,icc))

In [ ]:
boot.ci(results)